In [1]:
import numpy as np

In [90]:
class R_pca_nonconv:
    def __init__(self, D, beta=None):
        self.D = D.copy()
        self.L = np.zeros(self.D.shape)

        if beta:
            self.beta = beta
        else:
            self.beta = 1. / np.sqrt(np.max(self.D.shape))
            
        self.thr = self.beta * self.kth_singular_value(self.D, 0)
        self.S = self.hard_threshold(self.D, self.thr)

    @staticmethod
    def frobenius_norm(M):
        return np.linalg.norm(M, ord='fro')
    
    @staticmethod
    def kth_elem_safe(data, k):
        return 0.0 if k > data.size else data[k]
    
    def kth_singular_value(self, M, k):
        return self.kth_elem_safe(
            np.linalg.svd(M, full_matrices=False, compute_uv=False),
            k
        )
    
    @staticmethod
    def low_rank_approx(M, k):
        u, sigmas, vt = np.linalg.svd(M, full_matrices=False)
        k = min(k, sigmas.size)
        return u[:, :k] @ np.diag(sigmas[:k]) @ vt[:k, :]

    @staticmethod
    def hard_threshold(M, tau):
        return M * (np.abs(M) >= tau)

    def fit(self, rank=None, tol=None, verbose=True):
        Lk = self.L
        Sk = self.S
        
        m, n = np.min(self.D.shape), np.max(self.D.shape)

        if rank is None:
            rank = m
        
        if tol is None:
            tol = 1e-2

        
        #https://arxiv.org/pdf/1410.7660.pdf
        for k in range(rank):
            T = n * self.beta * self.kth_singular_value(self.D - Sk, 0) / tol            
            if T <= 1.0:
                T = 0
            else:
                T = 10 * int(np.log2(T))
                
            for t in range(T + 1):
                sigmas = np.linalg.svd(self.D - Sk, full_matrices=False, compute_uv=False)
                self.thr = self.beta * (
                    self.kth_elem_safe(sigmas, k + 1) + \
                    0.5 ** t * self.kth_elem_safe(sigmas, k)
                )
                Lk = self.low_rank_approx(self.D - Sk, k + 2)
                Sk = self.hard_threshold(self.D - Lk, self.thr)

            if verbose:
                err = self.frobenius_norm(self.D - Lk - Sk)
                print('stage: {0}, error: {1}'.format(k, err))

            if self.beta * self.kth_singular_value(Lk, k + 1) < tol / (2 * n):
                self.L = Lk
                self.S = Sk
                return Lk, Sk
        self.L = Lk
        self.S = Sk
        return Lk, Sk

In [103]:
test_arr = np.random.rand(20, 200)
test_rpca = R_pca_nonconv(test_arr)

In [104]:
test_l, test_s = test_rpca.fit(rank=8)

stage: 0, error: 14.745463078710278
stage: 1, error: 13.305664828330377
stage: 2, error: 11.882896776469845
stage: 3, error: 10.55842858187305
stage: 4, error: 9.648239733189738
stage: 5, error: 7.823570561174775
stage: 6, error: 6.84504553472256
stage: 7, error: 5.732362753180521


In [105]:
np.linalg.svd(test_l, compute_uv=False)

array([3.19518455e+01, 7.52727730e+00, 7.45770977e+00, 6.68630270e+00,
       6.42696999e+00, 6.17296067e+00, 5.29075143e+00, 4.41315175e+00,
       3.70141569e+00, 7.53023739e-15, 2.85302367e-15, 2.31670799e-15,
       1.79324141e-15, 1.74172272e-15, 1.59355057e-15, 1.40592743e-15,
       1.30525951e-15, 1.22350055e-15, 1.13896649e-15, 1.03120690e-15])

In [106]:
np.count_nonzero(test_s)

481

In [107]:
(np.abs(test_s) > 1e-6).sum()

481

In [110]:
test_l

array([[0.38219612, 0.21190067, 0.16481207, ..., 0.28736162, 0.47721196,
        0.37625956],
       [0.71076312, 0.57779748, 0.72635395, ..., 0.10628599, 0.41621997,
        0.29159198],
       [0.73711974, 0.53975325, 0.56489078, ..., 0.32547696, 0.74252961,
        0.72076683],
       ...,
       [0.41604765, 0.7096939 , 0.19265961, ..., 0.28951164, 0.72446488,
        0.71270645],
       [0.47872903, 0.63035299, 0.17164483, ..., 0.71186584, 1.13077862,
        0.68364561],
       [0.71021475, 0.65678981, 0.74708592, ..., 0.05166416, 0.80797025,
        0.35303444]])

In [111]:
test_s

array([[-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        , ...,  0.        ,
        -0.        , -0.        ],
       [ 0.        ,  0.        ,  0.36150735, ...,  0.        ,
         0.        , -0.        ],
       ...,
       [-0.        , -0.60946045,  0.        , ..., -0.        ,
         0.        ,  0.        ],
       [-0.        , -0.        ,  0.        , ...,  0.        ,
        -0.9430526 , -0.        ],
       [-0.        , -0.        , -0.        , ...,  0.86789919,
         0.        ,  0.        ]])